## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from numba import njit , jit
from tqdm.notebook import trange, tqdm

importing Jupyter notebook from Reading_data.ipynb
importing Jupyter notebook from Cleaning_data.ipynb


In [2]:
def nodes_and_edges(df):
    x = np.array(df.x)
    y = np.array(df.y)
    def car_to_cyl_cood(x, y):
        r = np.sqrt(x**2 + y**2)
        phi = np.arctan2(y, x)
        return r , phi
    
    r_hits , phi_hits = car_to_cyl_cood(x, y)
    
    df["phi"] = phi_hits
    df["r"] = r_hits

    z = np.array(df.z)    
    r = np.array(df.r)
    phi = np.array(df.phi)
    particles = np.array(df.particle_id)
    hit_id = np.array(df.hit_id)

    
    @njit(error_model='numpy')
    def func(x, y, r, phi, z, particles, hit_id):
        node_to_node_feat = []
        for i, idx in enumerate(range(len(phi))):
            for k, kdx in enumerate(range(len(phi))):
                dpdr = (phi[k]- phi[i])/(r[k]-r[i])
                dr = r[k]-r[i]
                z0 =  z[i] - r[i]*((z[k]-z[i])/(r[k]-r[i]))
                if abs(z0) <= 70 and dr > 0:
                    node_to_node_feat.append([(particles[i] == particles[k])*1, dpdr, idx, kdx, hit_id[idx], hit_id[kdx], dr, z[idx]
                                  , z[kdx], r[idx], r[kdx], phi[idx], phi[kdx], x[idx], x[kdx], y[idx], y[kdx], z0])
        return node_to_node_feat
    
    node_to_node_feat = func(x, y, r, phi, z, particles, hit_id) 
    
    node_to_node_feat = pd.DataFrame(node_to_node_feat, columns=['Y_k' , "dpdr" , 'node_1', 'node_2',"hit_id_node1","hit_id_node2","dr","z_node1","z_node2","r_node1","r_node2","phi_node1","phi_node2",
                            "x_node1","x_node2","y_node1","y_node2","z0"])
    return node_to_node_feat

In [3]:
def REWMH(data):#removing_edges_with_to_many_hits_ids
    data_sorted_True = data.loc[data.Y_k == 1]
    data_sorted_True = data_sorted_True.sort_values(by = ["hit_id_node1","dr"],ascending=True)
    data_True = data_sorted_True.drop_duplicates(subset = "hit_id_node1",ignore_index = True)
    data_sorted_False = data.loc[data.Y_k == 0]
    data_sorted_False = data_sorted_False.sort_values(by = ["hit_id_node1","dr"],ascending=True)
    data_False = data_sorted_False.drop_duplicates(subset = "hit_id_node1",ignore_index = True)
    True_and_false = pd.concat([data_True, data_False], ignore_index=True)
    return True_and_false

Reading data for 100 events:


  0%|          | 0/100 [00:00<?, ?it/s]

Cleaning data for 100 events:


  0%|          | 0/100 [00:00<?, ?it/s]

Finding nodes for all 100 events:


...

Splitting the data to 95 traning events:


  0%|          | 0/95 [00:00<?, ?it/s]

Splitting the data to 95 testing events:


  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: total: 3min 6s
Wall time: 3min 6s
